# Azure Machine Learning SDK for Python

## 1. Workspace

[Interactive authentication](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication#use-interactive-authentication)

### Create

In [ ]:
# from azureml.core import Workspace
# ws = Workspace.create(name='myworkspace',
#                       subscription_id='<azure-subscription-id>',
#                       resource_group='myresourcegroup',
#                       create_resource_group=True,
#                       location='eastus2'
#                      )

### Load

In [ ]:
from azureml.core import Workspace

# ws = Workspace.get(name="myworkspace", subscription_id='<azure-subscription-id>', resource_group='myresourcegroup')
ws = Workspace.from_config()
ws

## 2. Experiment

### Create

In [ ]:
from azureml.core.experiment import Experiment

experiment = Experiment(workspace=ws, name='test-experiment')
experiment

### List

In [ ]:
list_experiments = Experiment.list(ws)

list_experiments

### Get Runs

In [ ]:
list_runs = experiment.get_runs()

for run in list_runs:
    print(run.id)

### Tag

In [ ]:
experiment.tag("prod","phase-1-model-test")

## 3. Run

In [ ]:
run = experiment.start_logging()
run.id

experiment.tag("prod","phase-1-model-test")

In [ ]:
from azureml.core.run import Run

all_runs = Run.list(experiment)

for r in all_runs:
    print(r)

### Get run in details

In [ ]:
run_details = run.get_details()
run_details

In [ ]:
run.complete()

## Model

In [ ]:
from sklearn import svm
import joblib
import numpy as np

# customer ages
X_train = np.array([50, 17, 35, 23, 28, 40, 31, 29, 19, 62])
X_train = X_train.reshape(-1, 1)
# churn y/n
y_train = ["yes", "no", "no", "no", "yes", "yes", "yes", "no", "no", "yes"]

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(X_train, y_train)

joblib.dump(value=clf, filename="churn-model.pkl")

### Register Model

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="churn-model.pkl", model_name="churn-model-test")

### Download Model

In [ ]:
from azureml.core.model import Model
import os

modelv1 = Model(workspace=ws, name="churn-model-test")
modelv1.download(target_dir=os.getcwd(), exist_ok=True)
modelv1

### Delete Model

In [ ]:
model.delete()

## ComputeTarget, RunConfiguration, and ScriptRunConfig

### Train on Remote Compute

In [ ]:
%%writefile train.py

from sklearn import svm
import numpy as np
import joblib
import os

# customer ages
X_train = np.array([50, 17, 35, 23, 28, 40, 31, 29, 19, 62])
X_train = X_train.reshape(-1, 1)
# churn y/n
y_train = ["yes", "no", "no", "no", "yes", "yes", "yes", "no", "no", "yes"]

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(X_train, y_train)

os.makedirs("outputs", exist_ok=True)
joblib.dump(value=clf, filename="outputs/churn-model.pkl")

### RunConfiguration

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)

compute_config = RunConfiguration()
compute_config.target = "cpu-cc-training"
compute_config.amlcompute.vm_size = "STANDARD_DS3_V2"

### Dependencies

Training environment details are in the next cell.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
compute_config.environment.python.conda_dependencies = dependencies

### ScriptRunConfig

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=os.getcwd(), script="train.py", run_config=compute_config)
experiment = Experiment(workspace=ws, name="compute_target_test")
run = experiment.submit(config=script_run_config)
# run.wait_for_completion(show_output=True)

## Environment

In [ ]:
from azureml.core.environment import Environment

myenv = Environment(name="myenv")
myenv

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

conda_dep = CondaDependencies()

# Installs numpy version 1.17.0 conda package
conda_dep.add_conda_package("numpy==1.17.0")

# Installs pillow package
conda_dep.add_pip_package("pillow")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

myenv

## Model Deploy

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
from sklearn.externals import joblib


def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'churn-model.pkl')
    model = joblib.load(model_path)

def run(data):
    try:
        data = np.array(json.loads(data))
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [ ]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
model = run.register_model(model_name = "churn-model-test", model_path = "outputs/churn-model.pkl")

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "amlworkshop22", #change name to be unique
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service

In [ ]:
service